In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

Деревья классификации
--------------------------------------

In [ ]:
np.random.seed(43)

n = 250

mu1 = np.array([0.0,0])
mu2 = np.array([1.0,0])
sigma1 = 5.0 * np.diag(np.array([1.0, 1.0]))
sigma2 = 0.5 * np.diag(np.array([1.0, 1.0]))

x1 = np.random.multivariate_normal(mu1, sigma1, n)
x2 = np.random.multivariate_normal(mu2, sigma2, n)
x = np.vstack([x1, x2])
y = np.concatenate([np.full(x1.shape[0], 0), np.full(x2.shape[0], 1)])

plt.figure()
plt.scatter(*x1.T,s=2.5)
plt.scatter(*x2.T,s=2.5)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

In [ ]:
def flat_dict(x):
    if len(x) == 0:
        return dict()
    return {k: np.asarray([e[k] for e in x]) for k in x[0].keys()}

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate

max_depths = np.arange(1, 2 * np.log2(n))

scoring = {
    "auc":       "roc_auc",
    "accuracy":  "accuracy",
}

scores = []
for max_depth in max_depths:
    c = DecisionTreeClassifier(max_depth=max_depth)
    s = cross_validate(c, x, y.reshape(-1), cv=5, scoring=scoring, return_train_score=True)
    scores.append(s)

In [ ]:
scores = flat_dict(scores)

In [ ]:
plt.plot(max_depths, scores['train_accuracy'].mean(axis=1), '-*', label="train accuracy")
plt.plot(max_depths, scores['test_accuracy'].mean(axis=1), '-*', label="test accuracy")
plt.xlabel("Tree max depth")
plt.ylabel("Accuracy")
_ = plt.legend()

In [ ]:
x_grid = np.linspace(np.min(x), np.max(x), 2000)
xx, yy = np.meshgrid(x_grid, x_grid)
xx_test = np.stack((xx,yy), axis=-1).reshape(-1, 2)

c = DecisionTreeClassifier(random_state=0, max_depth=15)

c.fit(x_train, y_train)
pred = c.predict(xx_test).reshape(xx.shape)

x1_train = x_train[y_train == 0]
x2_train = x_train[y_train == 1]

plt.figure()
plt.xlim(-2,3)
plt.ylim(-2,3)
plt.contourf(xx, yy, pred, cmap="pink_r")
plt.scatter(*x1_train.T,s=2.5)
plt.scatter(*x2_train.T,s=2.5)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate

min_samples_leafs = np.arange(1, 20)

scoring = {
    "auc":       "roc_auc",
    "accuracy":  "accuracy",
}

scores = []
for min_samples_leaf in min_samples_leafs:
    c = DecisionTreeClassifier(min_samples_leaf=min_samples_leaf)
    s = cross_validate(c, x, y.reshape(-1), cv=5, scoring=scoring, return_train_score=True)
    scores.append(s)

In [ ]:
scores = flat_dict(scores)

In [ ]:
plt.plot(min_samples_leafs, scores['train_accuracy'].mean(axis=1), '-*', label="train accuracy")
plt.plot(min_samples_leafs, scores['test_accuracy'].mean(axis=1), '-*', label="test accuracy")
plt.xlabel("Leaf size")
plt.ylabel("Accuracy")
_ = plt.legend()

In [ ]:
c = DecisionTreeClassifier(random_state=0)

path = c.cost_complexity_pruning_path(x_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

In [ ]:
fig, ax = plt.subplots()
ax.plot(ccp_alphas[:-1], impurities[:-1], marker='o', drawstyle="steps-post")
ax.set_xlabel("Effective alpha")
_ = ax.set_ylabel("Total impurity of leaves")

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate

scoring = {
    "auc":       "roc_auc",
    "accuracy":  "accuracy",
}

scores = []
for ccp_alpha in ccp_alphas:
    c = DecisionTreeClassifier(ccp_alpha=ccp_alpha)
    s = cross_validate(c, x, y.reshape(-1), cv=5, scoring=scoring, return_train_score=True)
    scores.append(s)

In [ ]:
scores = flat_dict(scores)

In [ ]:
plt.plot(ccp_alphas, scores['train_accuracy'].mean(axis=1), '-*', label="train accuracy")
plt.plot(ccp_alphas, scores['test_accuracy'].mean(axis=1), '-*', label="test accuracy")
plt.xlabel("CCP alpha")
plt.ylabel("Accuracy")
_ = plt.legend()

Деревья классификации своими руками
-----------------------------------------------------

In [ ]:
from collections import namedtuple
from scipy import optimize

Leaf = namedtuple('Leaf', ('value', 'x', 'y'))
Node = namedtuple('Node', ('feature', 'value', 'impurity', 'left', 'right',))

class BaseDecisionTree:
    def __init__(self, x, y, max_depth=np.inf):
        self.x = np.atleast_2d(x)
        self.y = np.atleast_1d(y)
        self.max_depth = max_depth
        
        self.features = x.shape[1]
        
        self.root = self.build_tree(self.x, self.y)
    
    # Will fail in case of depth ~ 1000 because of limit of recursion calls
    def build_tree(self, x, y, depth=1):
        if depth > self.max_depth or self.criteria(y) < 1e-6:
            return Leaf(self.leaf_value(y), x, y)
        
        feature, value, impurity = self.find_best_split(x, y)
        
        left_xy, right_xy = self.partition(x, y, feature, value)
        left = self.build_tree(*left_xy, depth=depth + 1)
        right = self.build_tree(*right_xy, depth=depth + 1)
        
        return Node(feature, value, impurity, left, right)
    
    def leaf_value(self, y):
        raise NotImplementedError
    
    def partition(self, x, y, feature, value):
        i = x[:, feature] >= value
        j = np.logical_not(i)
        return (x[j], y[j]), (x[i], y[i])
    
    def _impurity_partition(self, value, feature, x, y):
        (_, left), (_, right) = self.partition(x, y, feature, value)
        return self.impurity(left, right)
    
    def find_best_split(self, x, y):
        best_feature, best_value, best_impurity = 0, x[0,0], np.inf
        for feature in range(self.features):
            if x.shape[0] > 2:
                x_interval = np.sort(x[:,feature])
                res = optimize.minimize_scalar(
                    self._impurity_partition, 
                    args=(feature, x, y),
                    bounds=(x_interval[1], x_interval[-1]),
                    method='Bounded',
                )
                assert res.success
                value = res.x
                impurity = res.fun
            else:
                value = np.max(x[:,feature])
                impurity = self._impurity_partition(value, feature, x, y)
            if impurity < best_impurity:
                best_feature, best_value, best_impurity = feature, value, impurity
        return best_feature, best_value, best_impurity
    
    # Can be optimized for given .criteria()
    def impurity(self, left, right):
        h_l = self.criteria(left)
        h_r = self.criteria(right)
        return (left.size * h_l + right.size * h_r) / (left.size + right.size)
    
    def criteria(self, y):
        raise NotImplementedError
        
    def predict(self, x):
        x = np.atleast_2d(x)
        y = np.empty(x.shape[0], dtype=self.y.dtype)
        for i, row in enumerate(x):
            node = self.root
            while not isinstance(node, Leaf):
                if row[node.feature] >= node.value:
                    node = node.right
                else:
                    node = node.left
            y[i] = node.value
        return y

In [ ]:
class MyDecisionTreeClassifier(BaseDecisionTree):
    def __init__(self, x, y, *args, random_state=None, **kwargs):
        y = np.asarray(y, dtype=int)
        self.random_state = np.random.RandomState(random_state)
        self.classes = np.unique(y)
        super().__init__(x, y, *args, **kwargs)
        
    def leaf_value(self, y):
        class_counts = np.sum(y == self.classes.reshape(-1,1), axis=1)
        m = np.max(class_counts)
        most_common = self.classes[class_counts == m]
        if most_common.size == 1:
            return most_common[0]
        return self.random_state.choice(most_common)
    
    def criteria(self, y):
        """Gini"""
        p = np.sum(y == self.classes.reshape(-1,1), axis=1) / y.size
        return np.sum(p * (1 - p))

In [ ]:
c = MyDecisionTreeClassifier(x_train, y_train, max_depth=5, random_state=0)
pred = c.predict(xx_test).reshape(xx.shape)

In [ ]:
x1_train = x_train[y_train == 0]
x2_train = x_train[y_train == 1]

plt.figure()
plt.xlim(-2,3)
plt.ylim(-2,3)
plt.contourf(xx, yy, pred, cmap="pink_r")
plt.scatter(*x1_train.T,s=2.5)
plt.scatter(*x2_train.T,s=2.5)

**Задание 4.1** Перед вами класс `BaseDecisionTree`. Используйте класс `MyDecisionTreeClassifier` в качестве примера, чтобы самостоятельно сделать простое дерево регрессии `MyDecisionTreeRegressor`.

Для задачи регрессии в качестве значения листа уместно взять среднее арифметическое значений всех точек содержащихся в узле, в качестве критерия - среднеквадратичное отклонение.

Используя синтетический пример с зависимостью (приведен ниже)
$$y = 2 x + 1 + \epsilon$$
удебитесь в работоспособности дерева регрессии, подберите наилучший параметр `max_depth`.

*Бонусные баллы* можно получить если реализовать функцию `impurity` более эффективно, упростив формулы для случая регрессии.

In [ ]:
class MyDecisionTreeRegressor(BaseDecisionTree):
    def __init__(self, x, y, *args, random_state=None, **kwargs):
        # код потерялся
    
    def leaf_value(self, y):
        # код потерялся
    
    def criteria(self, y):
        # код потерялся

In [ ]:
from sklearn.metrics import mean_squared_error

np.random.seed(42)

n = 250
x = np.random.uniform(0, 1, size=(n, 1))
y = 2 * x[:, 0] + 1
y = y + np.random.normal(0, 0.1, size=y.shape)

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

dtr = MyDecisionTreeRegressor(x_train, y_train, random_state=0)

plt.xlabel('true')
plt.ylabel('predicted')
plt.plot(y_train, dtr.predict(x_train), 'o', label='train')
plt.plot(y_test, dtr.predict(x_test), 'o', label='test')
plt.legend()

train_score = mean_squared_error(y_train, dtr.predict(x_train))
test_score = mean_squared_error(y_test, dtr.predict(x_test))

print("train score = {:.4f}".format(train_score))
print("test score = {:.4f}".format(test_score))

Случайный лес
------------------------

In [ ]:
np.random.seed(43)

n = 250

mu1 = np.array([0.0,0])
mu2 = np.array([1.0,0])
sigma1 = 5.0 * np.diag(np.array([1.0, 1.0]))
sigma2 = 0.5 * np.diag(np.array([1.0, 1.0]))

x1 = np.random.multivariate_normal(mu1, sigma1, n)
x2 = np.random.multivariate_normal(mu2, sigma2, n)
x = np.vstack([x1, x2])
y = np.concatenate([np.full(x1.shape[0], 0), np.full(x2.shape[0], 1)])

plt.figure()
plt.scatter(*x1.T,s=2.5)
plt.scatter(*x2.T,s=2.5)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

c = RandomForestClassifier(random_state=0, max_depth=15)

c.fit(x_train, y_train)
pred = c.predict(xx_test).reshape(xx.shape)

x1_train = x_train[y_train == 0]
x2_train = x_train[y_train == 1]

plt.figure()
plt.xlim(-2,3)
plt.ylim(-2,3)
plt.contourf(xx, yy, pred, cmap="pink_r")
plt.scatter(*x1_train.T,s=2.5)
plt.scatter(*x2_train.T,s=2.5)

**Задание 4.2** Используйте готовый класс `RandomForestClassifier` чтобы проверить как работает классификация с помощью метода случайного леса. С помощью метода кросс-валидации получите и постройте на графике зависимость точности (accuracy) (для учебного и тестового множеств) от максимальной глубины деревьев случайного леса. Используйте данные из примера про дерево классификации.

In [ ]:
# код потерялся

In [ ]:
scores = flat_dict(scores)

In [ ]:
plt.plot(max_depths, scores['train_accuracy'].mean(axis=1), '-*', label="train accuracy")
plt.plot(max_depths, scores['test_accuracy'].mean(axis=1), '-*', label="test accuracy")
plt.xlabel("Tree max depth")
plt.ylabel("Accuracy")
_ = plt.legend()

In [ ]:
import sklearn.metrics

def evaluate(c, x, y):
    y_pred = c.predict(x)
    if getattr(c, "decision_function", None):
        scores = c.decision_function(x)
    else:
        scores = c.predict_proba(x)[:,1]

    tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y, y_pred, labels=['h', 'g']).ravel()
    accuracy  = (tp + tn) / (tn + fp + fn + tp)
    precision = tp / (tp + fp)
    recall    = tp / (tp + fn)
    specificity = tn / (tn + fp)
    baccuracy = 0.5 * (specificity + recall)
    f1 = 2 * precision * recall / (precision + recall)
    
    print("Accuracy                  = {:.4f}".format(accuracy))
    print("Ballanced accuracy        = {:.4f}".format(baccuracy))
    print("F1                        = {:.4f}".format(f1))
    print("Precision (PPV)           = {:.4f}".format(precision))
    print("Recall (sensitivity, TPR) = {:.4f}".format(recall))
    print("Specificity (TNR, 1-FPR)  = {:.4f}".format(specificity))
    
    min_score, max_score = np.min(scores), np.max(scores)
    bins = np.linspace(min_score, max_score, 25)
    plt.figure()
    plt.hist(scores[y.reshape(-1) == 'h'], bins, alpha=0.5, label='Hadron (negative)')
    plt.hist(scores[y.reshape(-1) == 'g'], bins, alpha=0.5, label='Gamma (positive)')
    plt.xlabel("Decision function (value)")
    plt.ylabel("Frequency")
    plt.legend()
    
    tpr, fpr, _ = sklearn.metrics.roc_curve(y, scores, pos_label='g')
    auc = sklearn.metrics.roc_auc_score(y, scores)
    plt.figure()
    plt.plot(fpr, tpr)
    plt.title("Receiver operating characteristic")
    plt.xlabel("False positive rate")
    plt.ylabel("True positive rate")
    print("AUC                       = {:.4f}".format(auc))

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

names = ["length", "width", "size", "conc", "conc1", "asym", "m3long", "m3trans", "alpha", "dist", "class"]
data = pd.read_csv('magic04.csv', names=names)

x = np.asarray(data.iloc[:, :-1])
y = np.asarray(data.iloc[:, [-1]])

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

c = RandomForestClassifier(random_state=0)

c.fit(x_train, y_train.reshape(-1))

train_acc = c.score(x_train, y_train) # accuracy
test_acc = c.score(x_test, y_test)

evaluate(c, x_test, y_test)

**Задание 4.4**

Используя данные с черенковского телескопа из файла `magic04.csv` и метод кросс-валидации подберите максимальную глубину случайного леса таким образом, чтобы получить наилучший AUC при бинарной классификации методом случайного леса (`RandomForestClassifier`).
Постройте график зависимости AUC от глубины дерева.

In [ ]:
# код потерялся

In [ ]:
scores = flat_dict(scores)

np.set_printoptions(precision=4)
print("fit time = {}".format(scores['fit_time'].mean(axis=1)))
for s in scoring.keys():
    print("{} = {}".format(s, scores["test_{}".format(s)].mean(axis=1)))

In [ ]:
plt.plot(max_depths, scores['test_auc'].mean(axis=1),'-*')
plt.xlabel("Tree depth")
plt.ylabel("ROC AUC")

Адаптивный бустинг
----------------

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

c = AdaBoostClassifier(random_state=0, n_estimators=200)

c.fit(x_train, y_train)

train_acc = c.score(x_train, y_train) # accuracy
test_acc = c.score(x_test, y_test)

evaluate(c, x_test, y_test)

**Задание 4.5**

Используя данные с черенковского телескопа из файла `magic04.csv` и метод кросс-валидации подберите максимальную глубину дерева классификации, чтобы получить наилучший AUC при бинарной классификации методом адаптивного бустинга (`AdaBoostClassifier`).
Постройте график зависимости AUC от глубины дерева.

In [ ]:
# код потерялся

In [ ]:
scores = flat_dict(scores)

np.set_printoptions(precision=4)
print("fit time = {}".format(scores['fit_time'].mean(axis=1)))
for s in scoring.keys():
    print("{} = {}".format(s, scores["test_{}".format(s)].mean(axis=1)))

In [ ]:
plt.plot(max_depths, scores['test_auc'].mean(axis=1),'-*')
plt.xlabel("Tree depth")
plt.ylabel("ROC AUC")

Градиентный бустинг
------------------------------

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

c = GradientBoostingClassifier(random_state=0, n_estimators=200)

c.fit(x_train, y_train)

train_acc = c.score(x_train, y_train) # accuracy
test_acc = c.score(x_test, y_test)

evaluate(c, x_test, y_test)

**Задание 4.6**

Используя данные с черенковского телескопа из файла `magic04.csv` и метод кросс-валидации подберите максимальную глубину дерева классификации, чтобы получить наилучший AUC при бинарной классификации методом градиентного бустинга (`GradientBoostingClassifier`).
Постройте график зависимости AUC от глубины дерева.

In [ ]:
# код потерялся

In [ ]:
scores = flat_dict(scores)

np.set_printoptions(precision=4)
print("fit time = {}".format(scores['fit_time'].mean(axis=1)))
for s in scoring.keys():
    print("{} = {}".format(s, scores["test_{}".format(s)].mean(axis=1)))

In [ ]:
plt.plot(max_depths, scores['test_auc'].mean(axis=1),'-*')
plt.xlabel("Tree depth")
plt.ylabel("ROC AUC")

**Задание 4.7**

За это задание можно получить *бонусные баллы*, за три наилучших решения среди всей группы.

Используя один из четырех рассмотренных выше методов классификации с помощью деревьев, постройте бинарный классификатор для данных с черенковского телескопа из файла `magic04.csv` с наилучшим AUC для тестового набора данных. Можно подбирать любые гиперпараметры, которые окажется необходимо. Используйте функцию `evaluate`, чтобы нарисовать гистограмы классов и напечатать значения мер эффективности|.

Изолирующий лес
-----------------------------

In [ ]:
import numpy as np
import pandas as pd

wl = np.asarray([7.8636, 8.0485, 8.2286, 8.4043, 8.5758, 8.7436, 8.9078, 9.0686, 9.2262, 9.3809, 9.5328, 9.6820, 9.8286, 9.9728, 10.1148, 10.2545, 10.3922, 10.5279, 10.6616, 10.7935, 10.9237, 11.0521, 11.1790, 11.3042, 11.4280, 11.5503, 11.6711, 11.7907, 11.9089, 12.0258, 12.1415, 12.2560, 12.3693, 12.4816, 12.5927, 12.7028, 12.8118, 12.9199, 13.0269, 13.1330, 13.2382, 13.3425, 13.4459, 13.5485, 10.9929, 11.3704, 11.7357, 12.0899, 12.4339, 12.7687, 13.0948, 13.4131, 13.7239, 14.0278, 14.3252, 14.6166, 14.9022, 15.1825, 15.4576, 15.7280, 15.9937, 16.2551, 16.5123, 16.7656, 17.0151, 17.2610, 17.5034, 17.7425, 17.9784, 18.2113, 18.4412, 18.6682, 18.8925, 19.1142, 19.3334, 19.5500, 19.7643, 19.9763, 20.1861, 20.3937, 20.5992, 20.8026, 21.0041, 21.2037, 21.4014, 21.5973, 21.7914, 21.9838, 22.1745, 22.3636, 22.5511, 22.7371, 22.9216])
data = pd.read_csv('lrs.csv', header=None)

x = np.asarray(data.iloc[:, 11:54])
wl = wl[:x.shape[1]]

In [ ]:
plt.plot(wl, x[0,:], "-")
plt.title("IRAS/LRS spectra")
plt.ylabel("Intensity (units)")
plt.xlabel("Wavelength (um)")

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

pca = PCA(n_components=2, random_state=42)
z = pca.fit_transform(x)

c = IsolationForest(n_estimators=1000, contamination="auto")
c.fit(z)

pred = c.predict(z)

In [ ]:
_ = plt.scatter(*z.T, s=2.5, c=pred, cmap="coolwarm")

In [ ]:
plt.figure()
plt.plot(wl, pca.components_[0,:], "-")
plt.plot(wl, pca.components_[1,:], "-")
plt.title("IRAS/LRS spectra")
plt.ylabel("Intensity (units)")
_ = plt.xlabel("Wavelength (um)")

**Задание 4.8**

Используйте данные спектров космических объектов с ИК спутника из файла `lsr.csv` и метод изолирующего леса `IsolationForest`, чтобы найти 10 наболее отличных от остальных спектров. Распечатайте соответсвтвующие таким спектрам строки из исходного объекта `data`. Нарисуйте на графике самый необычный спектр.

In [ ]:
# код потерялся